In [6]:
import pandas as pd
import os
import re
import json


In [7]:
# -----------------------------------------------------
# 1. 경로 설정
# -----------------------------------------------------

# 노트북 기준 프로젝트 폴더
base_dir = r"C:\Users\73bib\Desktop\유혜원\제주한라대학교\[2025] 1학년 2학기\빅데이터 기초 및 실습\YT_ChannelGrowth_Engagement"

paths = {
    "BR": os.path.join(base_dir, "data", "raw", "youtube_trending_video_dataset", "BR_youtube_trending_data.csv"),
    "CA": os.path.join(base_dir, "data", "raw", "youtube_trending_video_dataset", "CA_youtube_trending_data.csv"),
    "DE": os.path.join(base_dir, "data", "raw", "youtube_trending_video_dataset", "DE_youtube_trending_data.csv"),
    "FR": os.path.join(base_dir, "data", "raw", "youtube_trending_video_dataset", "FR_youtube_trending_data.csv"),
    "GB": os.path.join(base_dir, "data", "raw", "youtube_trending_video_dataset", "GB_youtube_trending_data.csv"),
    "IN": os.path.join(base_dir, "data", "raw", "youtube_trending_video_dataset", "IN_youtube_trending_data.csv"),
    "JP": os.path.join(base_dir, "data", "raw", "youtube_trending_video_dataset", "JP_youtube_trending_data.csv"),
    "KR": os.path.join(base_dir, "data", "raw", "youtube_trending_video_dataset", "KR_youtube_trending_data.csv"),
    "MX": os.path.join(base_dir, "data", "raw", "youtube_trending_video_dataset", "MX_youtube_trending_data.csv"),
    "RU": os.path.join(base_dir, "data", "raw", "youtube_trending_video_dataset", "RU_youtube_trending_data.csv"),
    "US": os.path.join(base_dir, "data", "raw", "youtube_trending_video_dataset", "US_youtube_trending_data.csv"),
}

# JSON 폴더 경로
json_folder = os.path.join(base_dir, "data", "raw", "youtube_trending_video_categories")


In [8]:
# -----------------------------------------------------
# 2. 파일 누락 체크
# -----------------------------------------------------

missing = []

for country, path in paths.items():

    if not os.path.exists(path):
        missing.append((country, path))

if missing:
    print("누락된 CSV 파일:")

    for country, path in missing:
        print(f" - {country}: {path}")

    raise FileNotFoundError("일부 국가 CSV 파일이 누락되어 실행을 중단합니다.")

else:
    print("모든 국가 CSV 파일이 정상적으로 존재합니다.")


모든 국가 CSV 파일이 정상적으로 존재합니다.


In [9]:
# -----------------------------------------------------
# 3. 사용할 필수 컬럼 정의
# -----------------------------------------------------

use_cols = [
    "channelId", "video_id", "title",
    "publishedAt", "trending_date",
    "categoryId", "tags",
    "view_count", "likes", "comment_count"
]


In [10]:
# -----------------------------------------------------
# 4. 파일별 CSV 읽기 + 검증 + 전처리
# -----------------------------------------------------

yt_trending_list = []  # 리스트 초기화

for country, path in paths.items():

    # CSV 읽기
    try:
        df = pd.read_csv(path, low_memory=False, encoding="latin1")

    except Exception as e:
        print(f"CSV 형식 오류 발생 ({country}): {path}")
        raise ValueError(f"{country} CSV 파일을 읽는 중 오류 발생 → {str(e)}")

    # 필수 컬럼 누락 검사
    missing_cols = [col for col in use_cols if col not in df.columns]

    if missing_cols:
        raise ValueError(f"{country} CSV에서 필수 컬럼 누락: {missing_cols}")
        raise ValueError(f"{country} CSV에서 필수 컬럼이 누락되어 실행을 중단합니다.")

    print(f"{country}: 필수 컬럼 정상 확인")

    # 필요한 컬럼만 추출
    df = df[use_cols]

    # 국가 컬럼 추가
    df["country"] = country

    # 리스트에 추가
    yt_trending_list.append(df)

print("--------------------")
print("모든 CSV 컬럼 검증 완료")


BR: 필수 컬럼 정상 확인
CA: 필수 컬럼 정상 확인
DE: 필수 컬럼 정상 확인
FR: 필수 컬럼 정상 확인
GB: 필수 컬럼 정상 확인
IN: 필수 컬럼 정상 확인
JP: 필수 컬럼 정상 확인
KR: 필수 컬럼 정상 확인
MX: 필수 컬럼 정상 확인
RU: 필수 컬럼 정상 확인
US: 필수 컬럼 정상 확인
--------------------
모든 CSV 컬럼 검증 완료


In [11]:
# -----------------------------------------------------
# 5. 전체 병합 + 필터링
# -----------------------------------------------------

yt_trending_df = pd.concat(yt_trending_list, ignore_index=True)
print(f"병합 완료: {yt_trending_df.shape}")


병합 완료: (2905678, 11)


In [12]:
# ----------------------------------------------
# 6. 날짜 변환
# ----------------------------------------------

yt_trending_df["publishedAt"] = pd.to_datetime(yt_trending_df["publishedAt"], errors="coerce")
yt_trending_df["trending_date"] = pd.to_datetime(yt_trending_df["trending_date"], errors="coerce")

# 날짜 결측치 제거
yt_trending_df = yt_trending_df.dropna(subset=["publishedAt", "trending_date"])

In [13]:
# ----------------------------------------------
# 7. tags 파싱
# ----------------------------------------------

yt_trending_df["tags"] = yt_trending_df["tags"].fillna("")

def parse_tags(x):

    if x in ["[none]", "None", "none", ""]:
        return []
    
    return x.split("|")

yt_trending_df["tags_list"] = yt_trending_df["tags"].apply(parse_tags)
yt_trending_df["tags_count"] = yt_trending_df["tags_list"].apply(len)

In [14]:
# ----------------------------------------------
# 8. trending_days 계산
# ----------------------------------------------

yt_trending_df["trending_days"] = yt_trending_df.groupby("video_id")["video_id"].transform("count")

In [15]:
# ----------------------------------------------
# 9. 날짜 기반 파생 컬럼
# ----------------------------------------------

yt_trending_df["publish_month"] = yt_trending_df["publishedAt"].dt.month
yt_trending_df["publish_dayofweek"] = yt_trending_df["publishedAt"].dt.dayofweek
yt_trending_df["days_since_publish"] = (yt_trending_df["trending_date"] - yt_trending_df["publishedAt"]).dt.days

In [16]:
# ----------------------------------------------
# 10. 비율 기반 파생 컬럼
# ----------------------------------------------

yt_trending_df["like_ratio"] = yt_trending_df["likes"] / yt_trending_df["view_count"].replace(0, 1)
yt_trending_df["comment_ratio"] = yt_trending_df["comment_count"] / yt_trending_df["view_count"].replace(0, 1)
yt_trending_df["engagement_score"] = (yt_trending_df["likes"] + yt_trending_df["comment_count"]) / yt_trending_df["view_count"].replace(0, 1)

In [17]:
# ----------------------------------------------
# 11. categoryId → category_name 자동 매핑
# ----------------------------------------------

def load_category_map(json_path):

    if not os.path.exists(json_path):
        return {}

    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
        
    mapping = {}

    for item in data.get("items", []):
        cid = int(item["id"])
        name = item["snippet"]["title"]
        mapping[cid] = name

    return mapping

# 전체 국가 category JSON 자동 읽기
category_map = {}

if os.path.exists(json_folder):

    for country in paths.keys():
        json_path = os.path.join(json_folder, f"{country}_category_id.json")
        category_map.update(load_category_map(json_path))

# 매핑 적용
yt_trending_df["categoryId"] = yt_trending_df["categoryId"].astype(str).astype(int)
yt_trending_df["category_name"] = yt_trending_df["categoryId"].map(category_map).fillna("Unknown")

print("카테고리 매핑 로딩 완료 (총 개수:", len(category_map), ")")


카테고리 매핑 로딩 완료 (총 개수: 0 )


In [18]:
# -----------------------------------------------------
# 12. 파생 컬럼 검증
# -----------------------------------------------------

derived_cols = [
    "trending_days", "tags_list", "tags_count", "category_name",
    "publish_month", "publish_dayofweek", "days_since_publish",
    "like_ratio", "comment_ratio", "engagement_score"
]

missing_derived = [col for col in derived_cols if col not in yt_trending_df.columns]

if missing_derived:
    raise ValueError(f"파생 컬럼이 생성되지 않음: {missing_derived}")

print("파생 컬럼 검증 완료")


파생 컬럼 검증 완료


In [19]:
# -----------------------------------------------------
# 13. 저장 경로 자동 생성
# -----------------------------------------------------

# base_dir: 파일을 저장할 폴더 경로
# base_name: 파일명 prefix (예: 'youtube_trending_video_clean')
def get_next_version_file(base_dir, base_name):
    existing_files = os.listdir(base_dir)

    # 패턴: youtube_trending_video_clean_v3.parquet
    pattern = re.compile(rf"{base_name}_v(\d+)\.csv")
    versions = []

    for f in existing_files:
        match = pattern.match(f)

        if match:
            versions.append(int(match.group(1)))

    # 기존 파일이 있으면 max+1, 없으면 1
    next_version = max(versions) + 1 if versions else 1

    filename = f"{base_name}_v{next_version}.csv"
    return os.path.join(base_dir, filename)


In [ ]:
# -----------------------------------------------------
# 14. 저장
# -----------------------------------------------------

base_dir = r"C:\Users\73bib\Desktop\유혜원\제주한라대학교\[2025] 1학년 2학기\빅데이터 기초 및 실습\YT_ChannelGrowth_Engagement\data\processed"
base_name = "youtube_trending_video_clean"

save_path = get_next_version_file(base_dir, base_name)
yt_trending_df.to_csv(save_path, index=False)

print(f"저장 완료: {save_path}")
print(f"최종 데이터 형태: {yt_trending_df.shape}")


저장 완료: C:\Users\73bib\Desktop\유혜원\제주한라대학교\[2025] 1학년 2학기\빅데이터 기초 및 실습\YT_ChannelGrowth_Engagement\data\processed\youtube_trending_video_clean_v2.csv
최종 데이터 형태: (2905678, 21)
